In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
country_code = pd.read_csv('Country_data/countries_48_falcon.csv', header=None)
country_sameas = pd.read_csv('Country_data/wikidata_dbpedia.csv')

country_code.columns = ['Country Name', 'WikiData', 'DBpedia']
display(country_code.shape, country_code.head(3), country_sameas.shape, country_sameas.head(3))

(48, 3)

,Country Name,WikiData,DBpedia
0,American Samoa,http://www.wikidata.org/entity/Q16641,http://dbpedia.org/resource/American_Samoa
1,Bahamas,http://www.wikidata.org/entity/Q778,http://dbpedia.org/resource/The_Bahamas
2,Bolivia,http://www.wikidata.org/entity/Q750,http://dbpedia.org/resource/Bolivia


(216, 5)

,Country Code 2,Country Code 3,Country Name,WikiData,DBpedia
0,AW,ABW,Aruba,http://www.wikidata.org/entity/Q21203,http://dbpedia.org/resource/Aruba
1,AF,AFG,Afghanistan,http://www.wikidata.org/entity/Q889,http://dbpedia.org/resource/Afghanistan
2,AO,AGO,Angola,http://www.wikidata.org/entity/Q916,http://dbpedia.org/resource/Angola


In [3]:
missing_country = country_code.merge(country_sameas[['Country Name']], on='Country Name', how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only'].reset_index()
missing_country.drop(columns=['index', '_merge'], inplace=True)

### Extracting country code from Wikidata

In [4]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
missing_country['Country Code 2'] = ''
missing_country['Country Code 3'] = ''
for i, row in missing_country.iterrows():
    query ="""
    SELECT DISTINCT ?code2 ?code3 WHERE {
          """+'<'+row['WikiData']+'>'+""" p:P298 ?statement0 .
          """+'<'+row['WikiData']+'>'+""" p:P297 ?statement1 .
          ?statement0 ps:P298 ?code3 .
          ?statement1 ps:P297 ?code2 .
    }
    """
#     print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for r in results['results']['bindings']:
        missing_country.at[i, 'Country Code 2'] = str(r['code2']['value'])
        missing_country.at[i, 'Country Code 3'] = str(r['code3']['value'])
        break
#      <http://data.coypu.org/country/Bolivia> <http://www.w3.org/2002/07/owl#sameAs> <https://data.coypu.org/country/BOL>.
missing_country = missing_country.loc[missing_country['Country Code 3']!='']
display(missing_country.shape, missing_country.head(3))

(23, 5)

,Country Name,WikiData,DBpedia,Country Code 2,Country Code 3
0,Bahamas,http://www.wikidata.org/entity/Q778,http://dbpedia.org/resource/The_Bahamas,BS,BHS
2,Brunei,http://www.wikidata.org/entity/Q921,http://dbpedia.org/resource/Brunei,BN,BRN
3,Cape Verde,http://www.wikidata.org/entity/Q1011,http://dbpedia.org/resource/Cape_Verde,CV,CPV


In [5]:
country_sameas = pd.concat([country_sameas,missing_country])
country_sameas

,Country Code 2,Country Code 3,Country Name,WikiData,DBpedia
0,AW,ABW,Aruba,http://www.wikidata.org/entity/Q21203,http://dbpedia.org/resource/Aruba
1,AF,AFG,Afghanistan,http://www.wikidata.org/entity/Q889,http://dbpedia.org/resource/Afghanistan
2,AO,AGO,Angola,http://www.wikidata.org/entity/Q916,http://dbpedia.org/resource/Angola
3,AL,ALB,Albania,http://www.wikidata.org/entity/Q222,http://dbpedia.org/resource/Albania
4,AD,AND,Andorra,http://www.wikidata.org/entity/Q228,http://dbpedia.org/resource/Andorra
...,...,...,...,...,...
22,SY,SYR,Syria,http://www.wikidata.org/entity/Q858,http://dbpedia.org/resource/Syria
23,TW,TWN,Taiwan,http://www.wikidata.org/entity/Q865,http://dbpedia.org/resource/Taiwan
24,VE,VEN,Venezuela,http://www.wikidata.org/entity/Q717,http://dbpedia.org/resource/Venezuela
25,EH,ESH,Western Sahara,http://www.wikidata.org/entity/Q6250,http://dbpedia.org/resource/Western_Sahara


In [8]:
country_sameas.to_csv('Country_data/wikidata_dbpedia_v2.csv', index=None)

In [6]:
239-216

23

In [ ]:
protein_encoded_rows = []
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
for p in list_protein:
    query ="""
    SELECT DISTINCT ?nameLabel WHERE {
          ?item p:P352 ?statement0.
          ?statement0 (ps:P352) """+'"'+p+'"'+""".
          ?item p:P702 ?statement1.
          ?statement1 (ps:P702) ?name.
          ?name rdfs:label ?nameLabel.
          FILTER(LANG(?nameLabel) = "en").
    }
    """
    #print(query)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    try:
        results = sparql.query().convert()
    except:
        time.sleep(20)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
    if len(results['results']['bindings']) == 0:
        print(p, 'without encoded_by')
        continue

    for r in results['results']['bindings']:
        row = {'uniprot_id': p, 'encoded_by': str(r['nameLabel']['value'])}
        protein_encoded_rows.append(row)

#     row = {'uniprot_id': p, 'encoded_by': str(results['results']['bindings'][0]['nameLabel']['value'])}
protein_encoded = pd.DataFrame.from_dict(protein_encoded_rows)
protein_encoded = protein_encoded.merge(uniprot_df, how='inner')
display(protein_encoded.shape, protein_encoded.head())
# P0DMM9, P24462, P57059